# Greedy Learner - Step 2
Step 2: Optimization algorithm. Formally state the optimization problem where the objective function is the maximization of the cumulative expected margin over all the products. Design a greedy algorithm to optimize the objective function when all the parameters are known. The algorithm works as follows. At the beginning, every item is associated with the corresponding lowest price. Then, evaluate the marginal increase obtained when the price of a single product is increased by a single level, thus considering 5 potential different price configurations at every iteration, and choose the price configuration providing the best marginal increase (a price configuration specifies the price of every product). The algorithm stops when no new configuration among the 5 evaluated is better than the previous one. For instance, at the beginning, evaluate the 5 price configurations in which all the products are priced with the lowest price except for one product which is priced with the second lowest price. If all these price configurations are worse than the configuration in which all the products are priced with the lowest price, stop the algorithm and return the configuration with the lowest price for all the products. Otherwise, choose the best price configuration and re-iterate the algorithm. Notice that the algorithm monotonically increases the prices as well as the cumulative expected margin. Therefore, the algorithms cannot cycle. However, there is not guarantee that the algorithm will return the optimal price configuration. Develop the algorithm by Python.


In [ ]:
## Explain maybe with an image how the greedy learner class works
import numpy as np

### Load different configurations

In [ ]:
# Look at prove.ipynb

def load_config(file_path):
    return

### Run simulation for each evaluation method
In order to answer to the following question: how the evaluation method impacts the solution search?

In [ ]:
# For each config file choosen run the simulation with different evaluation method, that are GraphEvaluator, Baseline and Environment one
# Use markdown to explain how different evaluation methods works

#### Plot the results

In [ ]:
# Plot the expected reward using matplotlib in a bar chart (so in total 3 close bars for each configuaration)

### Run simulation with specific evaluation but different learner (BruteForce vs GreedyLearner)
In order to answer to the question, is the solution found by the Greedy the optimal one?

In [ ]:
# Fix the evaluator as GraphEvaluator and check how the optimal reward differ in the two classes

#### Plot the results

In [ ]:
# As before use bar charts (2 bars for the learners for each configuration)